# pyLectureMultiModalAnalysis

## Feature extraction from video segment

### Functions: 2
#### video2frame(), frame2features()

### Author: Stelios Karozis

## video2frame()

In [1]:
def video2frame(sec,folderVID,file,folderIMG):
    import cv2
    vidcap = cv2.VideoCapture(folderVID+file)
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(folderIMG+file+'_'+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames,folderIMG+file+'_'+str(count)+".jpg"

### test

In [ ]:
file='hd0456.mov'
sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second -> 2fps
count=1
success,a = video2frame(sec,'./',file,'./')
while success:
    count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success,a = video2frame(sec,'./',file,'./')

## frame2features

In [4]:
def frame2features(frame):
    import tensorflow as tf
    from keras.preprocessing import image
    from keras.applications.vgg16 import VGG16
    from keras.applications.vgg16 import preprocess_input
    import numpy as np

    model = VGG16(weights='imagenet', include_top=False)
    #model.summary()

    fr_path = frame
    fr = image.load_img(fr_path, target_size=(224, 224))
    fr_data = image.img_to_array(fr)
    fr_data = np.expand_dims(fr_data, axis=0)
    fr_data = preprocess_input(fr_data)

    vgg16_feature = model.predict(fr_data)

    vgg16_feature_np = np.array(vgg16_feature)
    feature = vgg16_feature_np.flatten()
    tf.reset_default_graph()
    return feature

### test

In [ ]:
frame2features('1.jpg')

## Combined test

In [ ]:
import pandas as pd
pathIn= '../data/'
index = pd.read_csv(pathIn+'index.csv', sep=';')

In [ ]:
os.path.basename(pathIn)


In [5]:
import os
#from os.path import isfile, join
import glob
import numpy as np
import pandas as pd
import pathlib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

pathIn= '../data/'
index = pd.read_csv(pathIn+'index.csv', sep=';')

frameRate = 0.5 #//it will capture image in each 0.5 second -> 2fps
ftr_array=[]
#files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]#for sorting the file names properly
#files.sort(key = lambda x: x[5:-4])
#for i in range(len(files)):

for f,s in index[['FILE','SEG']].values:
    pathIn= '../data/'+f+'/'
    print(pathlib.Path('pathIn').suffix)
    pathOut='../data/'+f+'/frames/'
    files=f+'_'+str(s)    
    files=glob.glob(pathIn+files+'*')[0]
    suffix=os.path.splitext(files)[1]
    files=f+'_'+str(s)+suffix
    ftr_fr=[]
    filename=pathIn + files
    print(filename)
    sec = 0
    count=1
    success,fr = video2frame(sec,pathIn,files,pathOut)
    ftr = frame2features(fr)
    ftr_fr.append(ftr)

    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success,fr = video2frame(sec,pathIn,files,pathOut)
        if success == True:
            print(fr)
            ftr = frame2features(fr)
            ftr_fr.append(ftr)
    ftr_fr = np.vstack(ftr_fr) 
    ftr_fr = np.average(ftr_fr, axis=0)
    ftr_array.append(ftr_fr)
ftr_array = np.vstack(ftr_array)


../data/speaker1/speaker1_0.mov
../data/speaker1/frames/speaker1_0.mov_2.jpg
../data/speaker1/frames/speaker1_0.mov_3.jpg
../data/speaker1/frames/speaker1_0.mov_4.jpg
../data/speaker1/frames/speaker1_0.mov_5.jpg
../data/speaker1/frames/speaker1_0.mov_6.jpg
../data/speaker1/frames/speaker1_0.mov_7.jpg

../data/speaker1/speaker1_1.mov
../data/speaker1/frames/speaker1_1.mov_2.jpg
../data/speaker1/frames/speaker1_1.mov_3.jpg
../data/speaker1/frames/speaker1_1.mov_4.jpg
../data/speaker1/frames/speaker1_1.mov_5.jpg

../data/speaker2/speaker2_0.mov
../data/speaker2/frames/speaker2_0.mov_2.jpg
../data/speaker2/frames/speaker2_0.mov_3.jpg
../data/speaker2/frames/speaker2_0.mov_4.jpg
../data/speaker2/frames/speaker2_0.mov_5.jpg
../data/speaker2/frames/speaker2_0.mov_6.jpg
../data/speaker2/frames/speaker2_0.mov_7.jpg

../data/speaker2/speaker2_1.mov
../data/speaker2/frames/speaker2_1.mov_2.jpg
../data/speaker2/frames/speaker2_1.mov_3.jpg
../data/speaker2/frames/speaker2_1.mov_4.jpg
../data/speak

In [30]:
ftr_df = pd.DataFrame(data=ftr_array)
df=index.copy()
df=pd.concat([df,ftr_df], axis=1)

In [33]:
df.to_pickle('../data/Video2Features.pkl')